In [50]:
"""
Title: Netflix Data Wrangling
Name: Salome Kungu
Cyber Shujaa ID: CS-DA03-26054
Program: DA1-2026
Date: 24th Jan 2026

Description:
This week’s assignment develops hands-on experience in data wrangling using
the Netflix dataset from Kaggle. The task focuses on data discovery, cleaning,
structuring, validation, and publishing a cleaned dataset.
"""


'\nTitle: Netflix Data Wrangling\nName: Salome Kungu\nCyber Shujaa ID: CS-DA03-26054\nProgram: DA1-2026\nDate: 24th Jan 2026\n\nDescription:\nThis week’s assignment develops hands-on experience in data wrangling using\nthe Netflix dataset from Kaggle. The task focuses on data discovery, cleaning,\nstructuring, validation, and publishing a cleaned dataset.\n'

In [4]:
# Step 1: Load the Dataset

#What:
#Import the Netflix CSV file into a Pandas DataFrame using pd.read_csv().

#Why:
#Loading the data into Pandas enables structured inspection, manipulation, and cleaning using efficient data-handling tools.

import pandas as pd
import numpy as np




In [47]:
#Load the Dataset

#What:
#Import the Netflix CSV file into a Pandas DataFrame using pd.read_csv().

#Why:
#Loading the data into Pandas enables structured inspection, manipulation, and cleaning using efficient data-handling tools.

import os
os.listdir('/kaggle/input')

!ls /kaggle/input/netflix-shows

netflix_titles.csv


In [6]:
#Step 2: Data discovery
#Load the Netflix dataset from the attached Kaggle dataset
#What:
#Use functions such as df.info(), df.shape, df.dtypes, df.isnull().sum(), and df.duplicated().sum() to explore the dataset.

#Why:
#This step helps identify data types, missing values, duplicates, and potential quality issues before any cleaning is performed.

filepath = '/kaggle/input/netflix-shows/netflix_titles.csv'
df = pd.read_csv(filepath)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [7]:
#Shape of dataset
#What:
#Display the number of rows and columns in the DataFrame using df.shape.

#Why:
#The shape attribute reveals the dataset's dimensions (rows, columns), helping assess data volume and structure for analysis planning.

print("Dataset Shape (Rows, Columns):", df.shape)

Dataset Shape (Rows, Columns): (8807, 12)


In [8]:
# List of all column names
print("Columns in the dataset:\n", df.columns.tolist())


Columns in the dataset:
 ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description']


In [9]:
# Data types of each column
print("Data types:\n", df.dtypes)


Data types:
 show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object


In [10]:
# Missing values per column
print("Missing values per column:\n", df.isnull().sum())


Missing values per column:
 show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64


In [11]:
# Duplicate rows
print("Number of duplicate rows:", df.duplicated().sum())


Number of duplicate rows: 0


In [12]:
#Step 3: Data Structuring

#What:
#Convert date_added to a datetime format. Split the duration column into numeric and unit components.
#Consider formatting or restructuring other columns where necessary.

#Why:
#Proper structuring ensures that columns are usable for analysis and prevents errors during filtering, grouping, or validation.

#Convert date_added to datetime
df['date_added'] = pd.to_datetime(df['date_added'], format='mixed', errors='coerce')


In [15]:
#Split duration into numeric value and unit

df[['duration_value', 'duration_unit']] = df['duration'].str.extract(r'(\d+)\s*(\w+)')


In [20]:
# Convert duration_value to numeric
df['duration_value'] = pd.to_numeric(df['duration_value'], errors='coerce')


In [21]:
# View results
df[['duration', 'duration_value', 'duration_unit']].head()


,duration,duration_value,duration_unit
0,90 min,90.0,min
1,2 Seasons,2.0,Seasons
2,1 Season,1.0,Season
3,1 Season,1.0,Season
4,2 Seasons,2.0,Seasons


In [22]:
#Step 4: Remove Duplicates and Unnecessary Columns

#What:
#Drop duplicate records. Remove columns such as description that are not required for analysis.

#Why:
#This reduces noise in the dataset and improves performance while focusing only on relevant attributes.

#Remove Duplicates
print("Duplicate rows before:", df.duplicated().sum())
df = df.drop_duplicates()
print("Duplicate rows after:", df.duplicated().sum())


Duplicate rows before: 0
Duplicate rows after: 0


In [23]:
#Drop Unnecessary Column
# This will not crash even if the column is already missing
df = df.drop(columns=['description'], errors='ignore')


In [24]:
#Step 5: Handle Missing Values (Imputation & Labeling)

#What:

#1. Infer missing director values using repeated director–cast relationships.
#2. Fill missing country values using director–country mappings.
#3. Assign “Not Given” to unresolved missing values in key fields.

#Why:
#Intelligent imputation preserves useful records while clearly distinguishing unknown data from valid values.

#Impute Missing Director Values Using Cast Relationships

# Create director-cast combination
df['dir_cast'] = df['director'].astype(str) + '---' + df['cast'].astype(str)


In [25]:
# Count frequent director-cast combinations
counts = df['dir_cast'].value_counts()
filtered_counts = counts[counts >= 3]


In [26]:
# Build director-cast dictionary
dir_cast_dict = {}
for item in filtered_counts.index:
    director, cast = item.split('---')
    dir_cast_dict[cast] = director


In [27]:
# Fill missing director values using cast
df.loc[df['director'].isna(), 'director'] = df.loc[df['director'].isna(), 'cast'].map(dir_cast_dict)


In [28]:
# Assign 'Not Given' to remaining missing directors
df['director'] = df['director'].fillna('Not Given')


In [30]:
#Step 6: Remove Records with Critical Missing Data

#What:
#Drop rows missing essential fields such as date_added, rating, or duration.

#Why:
#Records missing critical attributes cannot be reliably analyzed and may distort insights.

#Fill Missing Country Using Director-Country Mapping

# Create director-country mapping
director_country = (
    df.dropna(subset=['director', 'country'])
      .groupby('director')['country']
      .first()
      .to_dict()
)


In [31]:
# Fill missing country values
df.loc[df['country'].isna(), 'country'] = df.loc[df['country'].isna(), 'director'].map(director_country)


In [32]:
# Assign 'Not Given' to remaining missing countries
df['country'] = df['country'].fillna('Not Given')


In [33]:
#Step 7: Error and Logic Validation

#What:
#Check for logical inconsistencies, such as titles added to Netflix before their release year.

#Why:
#Logical validation ensures the dataset reflects real-world conditions and maintains analytical integrity.

#Handle Other Missing Values

# Fill missing cast
df['cast'] = df['cast'].fillna('Not Given')


In [34]:
# Drop rows with critical missing values
df = df.dropna(subset=['date_added', 'rating', 'duration'])


In [35]:
#Step 8: Final Validation and Cleanup

#What:
#1. Remove temporary columns created during wrangling.
#2. Confirm correct data types.
#3. Sample records for visual inspection.
#4. Reset the DataFrame index.

#Why:
#This step ensures the dataset is clean, consistent, and presentation-ready.

#Error and Logic Validation
#Check for date inconsistencies

# Count records where date_added is before release_year
invalid_dates = df['date_added'].dt.year < df['release_year']
sum(invalid_dates)


14

In [36]:
# View inconsistent records
df.loc[invalid_dates, ['title', 'date_added', 'release_year']]


,title,date_added,release_year
1551,Hilda,2020-12-14,2021
1696,Polly Pocket,2020-11-15,2021
2920,Love Is Blind,2020-02-13,2021
3168,Fuller House,2019-12-06,2020
3287,Maradona in Mexico,2019-11-13,2020
3369,BoJack Horseman,2019-10-25,2020
3433,The Hook Up Plan,2019-10-11,2020
4844,Unbreakable Kimmy Schmidt,2018-05-30,2019
4845,Arrested Development,2018-05-29,2019
5394,Hans Teeuwen: Real Rancour,2017-07-01,2018


In [37]:
# Confirm no remaining inconsistencies
sum(df['date_added'].dt.year < df['release_year'])


14

In [38]:
#Step 9: Export and Publish

#What:
#Save the cleaned dataset as a CSV file and publish the Kaggle Notebook with public access enabled.

#Why:
#Exporting and publishing ensure reproducibility, transparency, and compliance with submission requirements.

#Final Validation
#Remove temporary columns
df.drop(columns=['dir_cast'], inplace=True)


In [39]:
#Data type checks
df.dtypes


show_id                   object
type                      object
title                     object
director                  object
cast                      object
country                   object
date_added        datetime64[ns]
release_year               int64
rating                    object
duration                  object
listed_in                 object
duration_value           float64
duration_unit             object
dtype: object

In [40]:
#Business sanity rule (Netflix founded in 1997)
df[df['release_year'] < 1997]


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,duration_value,duration_unit
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",2021-09-24,1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies",125.0,min
22,s23,Movie,Avvai Shanmughi,K.S. Ravikumar,"Kamal Hassan, Meena, Gemini Ganesan, Heera Raj...",Not Given,2021-09-21,1996,TV-PG,161 min,"Comedies, International Movies",161.0,min
41,s42,Movie,Jaws,Steven Spielberg,"Roy Scheider, Robert Shaw, Richard Dreyfuss, L...",United States,2021-09-16,1975,PG,124 min,"Action & Adventure, Classic Movies, Dramas",124.0,min
42,s43,Movie,Jaws 2,Jeannot Szwarc,"Roy Scheider, Lorraine Gary, Murray Hamilton, ...",United States,2021-09-16,1978,PG,116 min,"Dramas, Horror Movies, Thrillers",116.0,min
43,s44,Movie,Jaws 3,Joe Alves,"Dennis Quaid, Bess Armstrong, Simon MacCorkind...",United States,2021-09-16,1983,PG,98 min,"Action & Adventure, Horror Movies, Thrillers",98.0,min
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8745,s8746,Movie,Willy Wonka & the Chocolate Factory,Mel Stuart,"Gene Wilder, Jack Albertson, Peter Ostrum, Roy...","United States, East Germany, West Germany",2020-01-01,1971,G,100 min,"Children & Family Movies, Classic Movies, Come...",100.0,min
8748,s8749,Movie,Winter of Our Dreams,John Duigan,"Judy Davis, Bryan Brown, Cathy Downes, Baz Luh...",Australia,2016-11-01,1981,NR,86 min,"Classic Movies, Dramas",86.0,min
8763,s8764,Movie,WWII: Report from the Aleutians,John Huston,Not Given,United States,2017-03-31,1943,TV-PG,45 min,Documentaries,45.0,min
8764,s8765,Movie,Wyatt Earp,Lawrence Kasdan,"Kevin Costner, Dennis Quaid, Gene Hackman, Dav...",United States,2020-01-01,1994,PG-13,191 min,Action & Adventure,191.0,min


In [41]:
#Sample records
df.sample(5)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,duration_value,duration_unit
5800,s5801,TV Show,Kulipari: An Army of Frogs,Not Given,"Mark Hamill, Keith David, Wendie Malick, Josh ...",United States,2016-09-02,2016,TV-Y7,1 Season,Kids' TV,1.0,Season
3790,s3791,Movie,Mere Pyare Prime Minister,Rakeysh Omprakash Mehra,"Anjali Patil, Om Kanojiya, Niteesh Wadhwa, Mak...",India,2019-05-31,2019,TV-MA,103 min,"Dramas, International Movies",103.0,min
8523,s8524,TV Show,The Super Mario Bros. Super Show!,Not Given,"Captain Lou Albano, Danny Wells, Jeannie Elias...","United States, Canada",2016-05-22,1989,TV-Y7,1 Season,Kids' TV,1.0,Season
8604,s8605,TV Show,Top 10 Secrets and Mysteries,Not Given,Not Given,"United Kingdom, United States, Czech Republic",2019-03-15,2018,TV-14,1 Season,"British TV Shows, Docuseries, Science & Nature TV",1.0,Season
2659,s2660,TV Show,Middleditch & Schwartz,Ryan Polito,"Thomas Middleditch, Ben Schwartz",United States,2020-04-21,2020,TV-MA,1 Season,TV Comedies,1.0,Season


In [43]:
#Reset index
df = df.reset_index(drop=True)


In [48]:
#Publish (Export Cleaned Dataset)
df.to_csv('/kaggle/working/cleaned_netflix.csv', index=False)


In [49]:
#verify the export explicitly to see if the operation succeeds.

import os

os.listdir('/kaggle/working')


['.virtual_documents', 'cleaned_netflix.csv']